# Why arrow?

This notebook explores the benefits or drawbacks of using the [arrow](https://arrow.apache.org) in-memory data format relative to other formats such as Pandas DataFrames.

In [ ]:
# set ipyflow reactive mode
%flow mode reactive